In [ ]:
import os
import sys
import pandas as pd
import numpy as np
from sklearn.svm import SVR
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.inspection import permutation_importance
from scipy.stats import loguniform
import matplotlib.pyplot as plt
import shap

# Define the current directory if __file__ is not available
current_dir = os.getcwd()  # Gets the current working directory
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))  # Moves one level up

# Add the parent directory to the Python path
sys.path.insert(0, parent_dir)

from save_and_compare_results import *


In [ ]:
# Define the path to the parent directory
data_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))  # Move one level up

# 1. Choose the phenotype

In [ ]:
phenotype = "YPD_doublingtime"
#phenotype = "YPDCUSO410MM_40h"

# 2. Preprocess the data

In [ ]:
X = pd.read_pickle(os.path.join(data_dir, f"data/X_matrix_proteins_nb_{phenotype}.pkl"))
Y = pd.read_pickle(os.path.join(data_dir, f"data/Y_matrix_proteins_nb_{phenotype}.pkl"))

# Preprocess the data
X = X.drop(columns=["Yeast_ID"]).fillna(0)
Y = Y.drop(columns=["Yeast_ID"]).fillna(Y.drop(columns=["Yeast_ID"]).mean())

# 3. Run the model

In [ ]:
param_distributions = {
    "C": loguniform(1e-3, 1e3),
    "epsilon": loguniform(1e-3, 1),
    "gamma": ["scale", "auto"],
    "kernel": ["linear", "rbf"]
}

svr = SVR()
random_search = RandomizedSearchCV(
    estimator=svr, 
    param_distributions=param_distributions,
    n_iter=50, 
    cv=3,
    verbose=1,
    random_state=42,
    n_jobs=-1
) 

random_search.fit(X, Y)

In [ ]:
# Get the best model and parameters
best_svr = random_search.best_estimator_
best_params = random_search.best_params_
print("\nBest hyperparameters:", best_params)

In [ ]:
# Evaluate the best model
y_pred = best_svr.predict(X)
r2 = r2_score(Y, y_pred)
mse = mean_squared_error(Y, y_pred)
print(f"\nR² Score: {r2:.4f}, Mean Squared Error: {mse:.4f}")

With number :  
best hyperparameters: {'C': 0.21481457181982688, 'epsilon': 0.0065169906111771725, 'gamma': 'scale', 'kernel': 'rbf'}  
R² Score: 0.2517, Mean Squared Error: 0.0375  

# 4. Model features importance

In [8]:
if best_params["kernel"] == "linear":
    print("Calculating feature importance for linear kernel...")
    
    # Feature importance is directly derived from the coefficients
    feature_importances = np.abs(best_svr.coef_[0])
    
    # Save and visualize feature importance
    save_feature_importance(
        features=X.columns,
        importance_scores=feature_importances,
        method="Coefficients",
        model_name="SVR"
    )
    
    # Display and plot top features
    feature_importances_df = pd.DataFrame({
        "Feature": X.columns,
        "Importance": feature_importances
    }).sort_values(by="Importance", ascending=False)
    
    top_features = feature_importances_df.head(10)
    print("\nTop 10 Features (Linear Kernel):")
    print(top_features)
    
    plt.figure(figsize=(10, 6))
    plt.barh(top_features["Feature"], top_features["Importance"], color="skyblue")
    plt.xlabel("Coefficient Magnitude")
    plt.ylabel("Feature")
    plt.title("Top 10 Features Impacting YPD Doubling Time (Linear Kernel)")
    plt.gca().invert_yaxis()
    plt.show()

In [ ]:
print("Displaying the 10 most important features...")

# Create a DataFrame with feature names and their importance scores
feature_importances_df = pd.DataFrame({
    "Feature": X.columns,
    "Importance": best_svr.feature_importances_
}).sort_values(by="Importance", ascending=False)

# Select the top 10 most important features
top_mutations = feature_importances_df.head(10)

# Display the top 10 features in the console
print("\nTop mutations impacting YPD doubling time:")
print(top_mutations)

# Plot the top 10 features with their importance scores
plt.figure(figsize=(10, 6))
plt.barh(top_mutations["Feature"], top_mutations["Importance"], color="skyblue")
plt.xlabel("Importance")
plt.ylabel("Mutation")
plt.title("Top 10 Mutations Impacting YPD Doubling Time")
plt.gca().invert_yaxis()
plt.show()

# 5. SHAP features importance

In [ ]:
print("Calculating SHAP values for SVR...")
explainer = shap.KernelExplainer(best_svr.predict, X)

# Calculate SHAP values
shap_values = explainer.shap_values(X, nsamples=100)

# Compute mean absolute SHAP values for feature importance
shap_mean_importance = np.abs(shap_values).mean(axis=0)

# Save SHAP feature importance
save_feature_importance(
    features=X.columns,
    importance_scores=shap_mean_importance,
    method="SHAP",
    model_name="SVR"
)

# Display and plot SHAP importance
feature_importances_df = pd.DataFrame({
    "Feature": X.columns,
    "Importance": shap_mean_importance
}).sort_values(by="Importance", ascending=False)

top_features = feature_importances_df.head(10)
print("\nTop 10 Features (SHAP):")
print(top_features)

plt.figure(figsize=(10, 6))
plt.barh(top_features["Feature"], top_features["Importance"], color="skyblue")
plt.xlabel("SHAP Value (Mean Absolute Importance)")
plt.ylabel("Feature")
plt.title("Top 10 Features Impacting YPD Doubling Time (SHAP)")
plt.gca().invert_yaxis()
plt.show()

In [ ]:
print("Generating SHAP plots...")
shap.summary_plot(shap_values, X, plot_type="bar")
shap.summary_plot(shap_values, X)

# 6. Permutation Importance

In [ ]:
print("Calculating permutation importance...")
perm_importance = permutation_importance(best_svr, X, Y.values.ravel(), n_repeats=10, random_state=42, scoring="r2")
feature_importances = perm_importance.importances_mean

# Save and visualize permutation importance
save_feature_importance(
    features=X.columns,
    importance_scores=feature_importances,
    method="Permutation",
    model_name="SVR"
)

# Display and plot permutation importance
feature_importances_df = pd.DataFrame({
    "Feature": X.columns,
    "Importance": feature_importances
}).sort_values(by="Importance", ascending=False)

top_features = feature_importances_df.head(10)
print("\nTop 10 Features (Permutation Importance):")
print(top_features)

plt.figure(figsize=(10, 6))
plt.barh(top_features["Feature"], top_features["Importance"], color="skyblue")
plt.xlabel("Permutation Importance (Mean)")
plt.ylabel("Feature")
plt.title("Top 10 Features Impacting YPD Doubling Time (Permutation Importance)")
plt.gca().invert_yaxis()
plt.show()